In [1]:
from collections import defaultdict
from sklearn import linear_model
import numpy
import math

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
### Question 1

In [ ]:
def getMaxLen(dataset):
    text_fields = ['review/text', 'text', 'review']
    lengths = [len(datum.get(field, '')) for datum in dataset for field in text_fields if field in datum]
    return max(lengths) if lengths else 0

In [ ]:
def featureQ1(datum, maxLen):
    text_fields = ['review/text', 'text', 'review']
    text_len = next((len(datum[field]) for field in text_fields if field in datum), 0)
    scaled_length = text_len / maxLen if maxLen > 0 else 0
    return [1, scaled_length]

In [ ]:
def Q1(dataset):
    maxLen = getMaxLen(dataset)
    rating_fields = ['rating', 'overall', 'stars', 'review/overall']
    
    X = [featureQ1(datum, maxLen) for datum in dataset]
    Y = [next((datum[field] for field in rating_fields if field in datum), 0) for datum in dataset]
    
    X, Y = numpy.array(X), numpy.array(Y)
    
    model = linear_model.LinearRegression(fit_intercept=False)
    model.fit(X, Y)
    
    theta = model.coef_
    MSE = numpy.mean((Y - model.predict(X)) ** 2)
    return theta, MSE

In [ ]:
### Question 2

In [ ]:
def featureQ2(datum, maxLen):
    text_fields = ['review/text', 'text', 'review']
    text_len = next((len(datum[field]) for field in text_fields if field in datum), 0)
    scaled_length = text_len / maxLen if maxLen > 0 else 0
    
    parsed_date = datum.get('parsed_date')
    weekday = parsed_date.weekday() if parsed_date else 0
    month = parsed_date.month if parsed_date else 1
    
    day_features = [1 if weekday == i else 0 for i in range(1, 7)]
    month_features = [1 if month == i else 0 for i in range(2, 13)]
    
    return [1, scaled_length] + day_features + month_features

In [ ]:
def Q2(dataset):
    maxLen = getMaxLen(dataset)
    rating_fields = ['rating', 'overall', 'stars', 'review/overall']
    
    X2 = [featureQ2(datum, maxLen) for datum in dataset]
    Y2 = [next((datum[field] for field in rating_fields if field in datum), 0) for datum in dataset]
    
    X2, Y2 = numpy.array(X2), numpy.array(Y2)
    
    model = linear_model.LinearRegression(fit_intercept=False)
    model.fit(X2, Y2)
    
    MSE2 = numpy.mean((Y2 - model.predict(X2)) ** 2)
    
    return X2, Y2, MSE2

In [ ]:
### Question 3

In [ ]:
def featureQ3(datum, maxLen):
    text_fields = ['review/text', 'text', 'review']
    text_len = next((len(datum[field]) for field in text_fields if field in datum), 0)
    scaled_length = text_len / maxLen if maxLen > 0 else 0
    
    parsed_date = datum.get('parsed_date')
    weekday = parsed_date.weekday() if parsed_date else 0
    month = parsed_date.month if parsed_date else 1
    
    return [1, scaled_length, weekday, month]

In [ ]:
def Q3(dataset):
    maxLen = getMaxLen(dataset)
    rating_fields = ['rating', 'overall', 'stars', 'review/overall']
    
    X3 = [featureQ3(datum, maxLen) for datum in dataset]
    Y3 = [next((datum[field] for field in rating_fields if field in datum), 0) for datum in dataset]
    
    X3, Y3 = numpy.array(X3), numpy.array(Y3)
    
    model = linear_model.LinearRegression(fit_intercept=False)
    model.fit(X3, Y3)
    
    MSE3 = numpy.mean((Y3 - model.predict(X3)) ** 2)
    
    return X3, Y3, MSE3

In [ ]:
### Question 4

In [ ]:
def Q4(dataset):
    n = len(dataset)
    train_data, test_data = dataset[:n//2], dataset[n//2:]
    
    maxLen = getMaxLen(train_data)
    rating_fields = ['rating', 'overall', 'stars', 'review/overall']
    
    X_train = [featureQ2(datum, maxLen) for datum in train_data]
    Y_train = [next((datum[field] for field in rating_fields if field in datum), 0) for datum in train_data]
    X_train, Y_train = numpy.array(X_train), numpy.array(Y_train)
    model2 = linear_model.LinearRegression()
    model2.fit(X_train, Y_train)
    
    X_train3 = [featureQ3(datum, maxLen) for datum in train_data]
    Y_train3 = [next((datum[field] for field in rating_fields if field in datum), 0) for datum in train_data]
    X_train3, Y_train3 = numpy.array(X_train3), numpy.array(Y_train3)
    model3 = linear_model.LinearRegression()
    model3.fit(X_train3, Y_train3)
    
    X_test = [featureQ2(datum, maxLen) for datum in test_data]
    Y_test = [next((datum[field] for field in rating_fields if field in datum), 0) for datum in test_data]
    X_test, Y_test = numpy.array(X_test), numpy.array(Y_test)
    
    X_test3 = [featureQ3(datum, maxLen) for datum in test_data]
    Y_test3 = [next((datum[field] for field in rating_fields if field in datum), 0) for datum in test_data]
    X_test3, Y_test3 = numpy.array(X_test3), numpy.array(Y_test3)
    
    test_mse2 = numpy.mean((Y_test - model2.predict(X_test)) ** 2)
    test_mse3 = numpy.mean((Y_test3 - model3.predict(X_test3)) ** 2)
    
    return test_mse2, test_mse3

In [ ]:
### Question 5

In [ ]:
def featureQ5(datum):
    text_fields = ['review/text', 'text', 'review']
    text_len = next((len(datum[field]) for field in text_fields if field in datum), 0)
    return [1, text_len]

In [ ]:
def Q5(dataset, feat_func):
    rating_fields = ['rating', 'overall', 'stars', 'review/overall']
    y = [1 if next((datum[field] for field in rating_fields if field in datum), 0) >= 4 else 0 for datum in dataset]
    
    X = [feat_func(datum) for datum in dataset]
    X, y = numpy.array(X), numpy.array(y)
    
    model = linear_model.LogisticRegression(class_weight='balanced', random_state=42)
    model.fit(X, y)
    
    y_pred = model.predict(X)
    
    TP = numpy.sum((y == 1) & (y_pred == 1))
    TN = numpy.sum((y == 0) & (y_pred == 0))
    FP = numpy.sum((y == 0) & (y_pred == 1))
    FN = numpy.sum((y == 1) & (y_pred == 0))
    
    BER = 0.5 * (FP / (FP + TN) + FN / (FN + TP)) if (FP + TN) > 0 and (FN + TP) > 0 else 0.5
    
    return TP, TN, FP, FN, BER

In [ ]:
### Question 6

In [ ]:
def Q6(dataset):
    rating_fields = ['rating', 'overall', 'stars', 'review/overall']
    y = [1 if next((datum[field] for field in rating_fields if field in datum), 0) >= 4 else 0 for datum in dataset]
    X = [featureQ5(datum) for datum in dataset]
    X, y = numpy.array(X), numpy.array(y)
    
    model = linear_model.LogisticRegression(class_weight='balanced', random_state=42)
    model.fit(X, y)
    
    y_proba = model.predict_proba(X)[:, 1]
    
    K_values = [1, 100, 1000, 10000]
    precs = []
    
    for K in K_values:
        top_k_indices = numpy.argsort(y_proba)[-K:]
        top_k_labels = y[top_k_indices]
        precision_k = numpy.sum(top_k_labels) / K if K > 0 else 0
        precs.append(precision_k)
    
    return precs

In [ ]:
### Question 7

In [ ]:
def featureQ7(datum):
    text_fields = ['review/text', 'text', 'review']
    text_len = next((len(datum[field]) for field in text_fields if field in datum), 0)
    
    style = datum.get('beer/style', 'Unknown')
    common_styles = ['IPA', 'Stout', 'Porter', 'Lager', 'Ale', 'Wheat', 'Pilsner', 'Saison']
    style_features = [1 if style in common_style else 0 for common_style in common_styles]
    
    abv = datum.get('beer/ABV', 0)
    
    appearance = datum.get('review/appearance', 0)
    aroma = datum.get('review/aroma', 0)
    taste = datum.get('review/taste', 0)
    palate = datum.get('review/palate', 0)
    
    log_length = numpy.log(text_len + 1)
    sqrt_length = numpy.sqrt(text_len)
    
    return [1, text_len, abv, appearance, aroma, taste, palate, log_length, sqrt_length] + style_features